<a href="https://colab.research.google.com/github/AnnweshaAdhikari/YouTube-video-transcription-and-summarization/blob/main/YouTube_video_transcription_and_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Downloading audio from a youtube video

In [ ]:
!pip install yt_dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 16.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [ ]:
!pip install huggingsound

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 79.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 103.8 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.3.0+cu121
    Uninstalling torch-2.3.0+cu121:
      Successfully uninstalled torch-2.3.0+cu121
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling 

In [ ]:
import yt_dlp

url = "https://youtu.be/OxGsU8oIWjY"  # "How An Infinite Hotel Ran Out Of Room" by Veritasium

ydl_opts = {
    'format': 'bestaudio/best',
    'outtmpl': 'youtubeaudio',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'wav',
        'preferredquality': '192',
    }],
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([url])

[youtube] Extracting URL: https://youtu.be/OxGsU8oIWjY
[youtube] OxGsU8oIWjY: Downloading webpage
[youtube] OxGsU8oIWjY: Downloading ios player API JSON
[youtube] OxGsU8oIWjY: Downloading m3u8 information
[info] OxGsU8oIWjY: Downloading 1 format(s): 251
[download] Destination: youtubeaudio
[download] 100% of    5.85MiB in 00:00:00 at 18.66MiB/s  
[ExtractAudio] Destination: youtubeaudio.wav
Deleting original file youtubeaudio (pass -k to keep)


## Using Automatic Speech Recognition with HuggingSound to generate a transcript

In [ ]:
from huggingsound import SpeechRecognitionModel

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
model = SpeechRecognitionModel("jonatasgrosman/wav2vec2-large-xlsr-53-english", device=device)

INFO:huggingsound.speech_recognition.model:Loading model...
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Performing audio chunking to avoid out-of-memory error

In [ ]:
import librosa
input = "/content/youtubeaudio.wav"
print(librosa.get_samplerate(input))

48000


In [ ]:
stream = librosa.stream(
    input,
    block_length=60,
    frame_length=48000,
    hop_length=48000)

In [ ]:
import soundfile as sf

for i,speech in enumerate(stream):
  sf.write(f"audio{i}.wav", speech, samplerate=48000)

## Performing the transcription

In [ ]:
audio_path = []
for k in range(i+1):
  audio_path.append(f"/content/audio{k}.wav")
audio_path

['/content/audio0.wav',
 '/content/audio1.wav',
 '/content/audio2.wav',
 '/content/audio3.wav',
 '/content/audio4.wav',
 '/content/audio5.wav',
 '/content/audio6.wav']

In [ ]:
transcripts = model.transcribe(audio_path)

100%|██████████| 7/7 [00:20<00:00,  2.87s/it]


In [ ]:
transcripts

[{'transcription': "imagine there's a hotel with infinite rooms they one two threefor and so on foreverthis is the hilbert hotel and you are the managerit might seem like you could accommodate anyone who ever shows up but there is a limit a way to exceed even the infinity of rooms at the hilbert hotelto start let say only one person is allowed in each room and all the rooms are full there are an infinite number of people in an infinite number of rooms theone  shows up and they want a rom but all the rooms are occupied so what should you dolesser manager might turn them away but you know about infinity so youget on pel all the guests to move down a room so the person in room one moves to roo to the onner-rom two moves to room three and so on down the line and now you can put the new guest",
  'start_timestamps': [2380,
   2460,
   2560,
   2660,
   2740,
   2780,
   2820,
   2900,
   3020,
   3040,
   3100,
   3120,
   3140,
   3180,
   3200,
   3240,
   3280,
   3300,
   3340,
   3400,

In [ ]:
actual_transcript = " "
for item in transcripts:
  actual_transcript += actual_transcript + "".join(item['transcription'])
actual_transcript

"  imagine there's a hotel with infinite rooms they one two threefor and so on foreverthis is the hilbert hotel and you are the managerit might seem like you could accommodate anyone who ever shows up but there is a limit a way to exceed even the infinity of rooms at the hilbert hotelto start let say only one person is allowed in each room and all the rooms are full there are an infinite number of people in an infinite number of rooms theone  shows up and they want a rom but all the rooms are occupied so what should you dolesser manager might turn them away but you know about infinity so youget on pel all the guests to move down a room so the person in room one moves to roo to the onner-rom two moves to room three and so on down the line and now you can put the new guest  imagine there's a hotel with infinite rooms they one two threefor and so on foreverthis is the hilbert hotel and you are the managerit might seem like you could accommodate anyone who ever shows up but there is a limi

## Finally, summarizing the transcript text of the YouTube video

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Chunking the text before summarizing

In [ ]:
num_chunks = int(len(actual_transcript)/2000)
summarized_text = []
for i in range(0,num_chunks+1):
  start = 0
  start = i*1000
  end = (i+1)*1000
  out = summarizer(actual_transcript[start:end],min_length = 5,max_length = 100)
  out = out[0]
  out = out['summary_text']
  summarized_text.append(out)
print(summarized_text)

[" Imagine there's a hotel with infinite rooms they one two threefor and so on forever this is the hilbert hotel and you are the manager .", ' A way to exceed even the infinity of rooms at the hilbert hotel .', " There is a limit a way to exceed even the infinity of rooms at the hilbert hotel . The hotel is really starting to feel like it can fit everybody and that's the beauty of infinity .", " Imagine there's a hotel with infinite rooms they one two threefor and so on forever this is the hilbert hotel and you are the manager .", ' If a bus shows up with a hundred people you know what to do with a finite number of people but what do you do with infinite people? "That\'s the beauty of infinity it goes on foreverthen all of a sudden more infinite busss showunotot jst one or two butan infinite number of infinite buses w', " Each person gets a unique identifier which is accombidation of their vehicle and their position in itsh you assign the rooms will start in the top-left corner and dra

In [ ]:
def wrap_list(input_list, line_length=75):
    for item in input_list:
        words = item.split()
        current_line = ""
        for word in words:
            if len(current_line) + len(word) + 1 > line_length:
                print(current_line)
                current_line = word
            else:
                if current_line:
                    current_line += " "
                current_line += word
        if current_line:
            print(current_line)
        print()

wrap_list(summarized_text)

Imagine there's a hotel with infinite rooms they one two threefor and so on
forever this is the hilbert hotel and you are the manager .

A way to exceed even the infinity of rooms at the hilbert hotel .

There is a limit a way to exceed even the infinity of rooms at the hilbert
hotel . The hotel is really starting to feel like it can fit everybody and
that's the beauty of infinity .

Imagine there's a hotel with infinite rooms they one two threefor and so on
forever this is the hilbert hotel and you are the manager .

If a bus shows up with a hundred people you know what to do with a finite
number of people but what do you do with infinite people? "That's the
beauty of infinity it goes on foreverthen all of a sudden more infinite
busss showunotot jst one or two butan infinite number of infinite buses w

Each person gets a unique identifier which is accombidation of their
vehicle and their position in itsh you assign the rooms will start in the
top-left corner and draw a line the zigzag